# 특정 class에서 특정 value 뽑기

In [23]:
import weaviate
import json


def get_all_data_in_specific_value(class_name,
                                   return_values:list,
                                   path_value,
                                    valuestring):
    client = weaviate.Client("http://115.71.28.105:2448") 
    class_name=class_name.capitalize()
    try:
        result = (
            client.query
            .get(class_name, return_values)
            .with_where({
                "path": [path_value],
                "operator": "Equal",
                "valueString": valuestring
            })
            .do()
        )

        if result and "data" in result and "Get" in result["data"]:
            objects = result["data"]["Get"][class_name]
            print(f"results: {len(objects)}")
            # for obj in objects:
            #     print(json.dumps(obj, indent=2, ensure_ascii=False))
            return objects
        else:
            print(f"no result abour {class_name} - {path_value} - {valuestring}")
            return []

    except Exception as e:
        print(f"Error: {str(e)}")
        return []

if __name__ == "__main__":
    results = get_all_data_in_specific_value("stt_test",
                                             ["vid_title", "text", "end"],
                                             "vid_title",
                                            "고윤정처럼_알람_맞추는_사람_손_들어___‘인간샤넬’이_평생_입고_싶은_컬러_한_가지는?__언더더씬___ELLE_KOREA")
# client.close()

results: 3


/tmp/ipykernel_7442/886462868.py:9: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client("http://115.71.28.105:2448")


In [24]:
"\n".join(list(map(lambda x: x["text"], results)))

'하나 둘 셋 안녕하세요 엘르 고윤정입니다 여기는 샤넬과 함께하는 엘르 디에디션 촬영장입니다 제가 초록색을 좋아해서 평소보다 유독 컨디션이 좋았던 것 같은 느낌이 듭니다 이게 크루즈 컬렉션 옷이라서 얼마 전에 갔던 마르세유에 다시 한번 가보고 싶습니다 평소에 안하던 걸 해보고 싶어요 길 가다가 꽃 같은 걸 산다던가 바닷가에 멈춰서 바람 한번 쐬고 그런 여유있고 낭만적인 걸 하고 오고 싶습니다 블랙? 편하게 자주 손이 가는 것 같아요 초록색 옷은 거의 없는 것 같아요 도전해 보는 것도 좋을 것 같다고 생각이 듭니다 잠이요\n잠 잘 시간이 별로 없어서 어떻게 하면 효율적으로 적은 수면시간으로 최대 효과를 끌어낼 수 있을지 고민에 푹 빠져있는 것 같아요 살면서요 한번 잠들면 절대 안 깨거든요 화장실도 안 가고 근데 그렇다고 그렇게 오래 자지는 않아요 13시간? 긴 건 아 13시간 참 아 13시간 참 루틴이 거의 없는 것 같은데 자기 전에 알람 맞추는 거 저 3분마다 5번 정도 맞춰요 그러면 5번째에 깨더라고요 피곤해요 근데 일어나서 가야 되긴 하니까 어제 많이 잤어요 어제 5시간 잤어요 클래식 샤넬이 갖고 있는 브랜드 이름이라는 거 이상의 가치가 이상의 가치가 있다고 생각이 들어서 샤넬은 타임리스한 브랜드 같다는 생각이 들어서 샤넬은 타임리스한 브랜드 같다는 생각이 들어서 다 너무 예뻤는데 잘 안 잤어요 다 너무 예뻤는데 잘 안 잤어요 잘 안 잤어요 피곤해요 근데 일어나서 가야 되긴 하니까 어제 많이 잤어요 어제 5시간 잤어요 클래식 샤넬이 갖고 있는 어떤 브랜드 이름이라는 거 이상의 가치가 있다고 생각이 들어서\n샤넬은 타임리스한 브랜드 같다는 생각이 듭니다 다 너무 예뻤는데 처음에 입었던 트위드 투피스 셋업인데 후드가 달린 착장이 있어요 샤넬 하면 좀 클래식하고 우아하고 그런 이미지가 우선적으로 떠오르는데 후드가 들어가니까 되게 캐주얼하고 신선한 느낌도 들어서 가장 기억에 남는 것 같습니다 오늘 이렇게 엘르와 함께 촬영을 해봤는데요 앞으로 나올 엘르 화보도 그리

# DB 내 전체 class data 뽑기

In [35]:
import weaviate
import json
from typing import List, Dict, Any, Optional

def load_all_class_data(
    class_name: str,
    fields: List[str] = None,
    batch_size: int = 100
) -> List[Dict[str, Any]]:

    client = weaviate.Client("http://115.71.28.105:2448") 

    if fields is None:
        try:
            schema = client.schema.get(class_name)
            fields = [prop['name'] for prop in schema['properties']]
        except Exception as e:
            print(f"스키마 로딩 중 오류 발생: {str(e)}")
            fields = ["vid_title", "content"] 

    fields.append("_additional {id}")

    all_objects = []
    offset = 0
    
    try:
        while True:
            result = (
                client.query
                .get(class_name, fields)
                .with_limit(batch_size)
                .with_offset(offset)
                .do()
            )

            if (not result or 
                "data" not in result or 
                "Get" not in result["data"] or 
                not result["data"]["Get"][class_name]):
                break

            batch_objects = result["data"]["Get"][class_name]
            all_objects.extend(batch_objects)

            print(f"로드된 데이터: {len(all_objects)}개")

            if len(batch_objects) < batch_size:
                break

            offset += batch_size

        print(f"\n총 {len(all_objects)}개의 데이터를 로드했습니다.")
        return all_objects

    except Exception as e:
        print(f"데이터 로딩 중 오류 발생: {str(e)}")
        return all_objects

def save_to_json(data: List[Dict[str, Any]], filename: str):
    """데이터를 JSON 파일로 저장"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"\n데이터가 {filename}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {str(e)}")

if __name__ == "__main__":
    CLASS_NAME = "Captioning_retrieval"  

    # 특정 필드만 가져오기
    # FIELDS = ["vid_title", "content"]
    # data = load_all_class_data(CLASS_NAME, fields=FIELDS)
    
    # 모든 필드 가져오기
    data = load_all_class_data(CLASS_NAME)

    # if data:
    #     save_to_json(data, f"{CLASS_NAME}_data.json")

로드된 데이터: 8개

총 8개의 데이터를 로드했습니다.


/tmp/ipykernel_7442/1278357656.py:11: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client("http://115.71.28.105:2448")


In [36]:
data

[{'_additional': {'id': '0796e0da-3292-45ad-aea7-9b43668e8a40'},
  'caption': '이 이미지에는 한 여성이 주요한 역할을 합니다. 그녀는 흰색과 검정색의 체크 패턴의 셔츠를 입고 있으며, 목에 목걸이를 착용하고 있습니다. 그녀는 빈티지 스타일의 테이블 위에 있는 검정색 상자 위에 서 있습니다. \n\n그녀의 뒤에는 벽이 있으며, 그 벽에는 "ASK ME ANYTHING"이라는 텍스트가 붙어 있습니다. 또한, 그녀의 옆에는 "줄거움"과 "책임감"이라는 텍스트가 붙어 있습니다. \n\n그녀의 옆에는 테이블 위에 여러 종류의 종이가 놓여져 있습니다. 그녀의 앞에는 테이블 위에 있는 검정색 상자가 놓여져 있습니다. \n\n이 모든 요소는 그녀의 주변 환경을 보여주며, 그녀는 이 환경에서 어떤 역할을 하는지에 대한 정보를 제공하고 있습니다.',
  'description': '#ELLEaskmeanything #고윤정',
  'time': '0:05:00',
  'url': 'https://youtu.be/Pb2WhqgUWl8?si=-qb_oyJeOknaa_aQ',
  'vid_title': "환혼_2_에서_이재욱과_오열한_바로_그_장면!_'희수'_vs_'낙수'_고윤정의_선택은?_고윤정의_ASK_ME_ANYTHING_____ELLE_KOREA"},
 {'_additional': {'id': '08e2d961-143c-4755-9d0a-e68be06c77aa'},
  'caption': "이 이미지에는 한 여성이 주로 주목받고 있습니다. 그녀는 검정과 화이트의 체크 패턴을 한 드레스를 입고 있으며, 목에 목걸이를 착용하고 있습니다. 그녀는 오른손으로 녹색 종이를 들고 있으며, 왼손에는 엘레가 쓰여 있는 노란색 종이를 들고 있습니다. 그녀의 앞에는 'ASK ME ANYTHING'이라는 텍스트가 있으며, 그녀의 옆에는 'Q4'라는 텍스트가 있습니다. 그녀의 옆에는 'ELLE'이라는 텍스트가 있으며, 그녀의 옆에는 '

In [37]:
def time_to_seconds(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    return time_str
    
sorted(data, key=lambda x: float(time_to_seconds(x['time'])) if x['time'] else float('inf'))

[{'_additional': {'id': '0f3d3178-178a-453a-adec-fe69405d15dd'},
  'caption': '이 이미지에는 한 여성이 주요한 역할을 하고 있습니다. 그녀는 흰색과 검은색의 체크 패턴의 재킷을 입고 있으며, 목에 목걸이를 착용하고 있습니다. 그녀는 빈 테이블 위에 있는 블랙색 상자 앞에 서 있습니다. \n\n테이블 위에는 여러 가지 종류의 종이가 퍼져 있습니다. 그 중 하나는 녹색, 다른 하나는 파란색, 그리고 마지막 하나는 분홍색입니다. 이 종이들은 테이블 위에 퍼져 있으며, 그 중 일부는 그녀의 앞에, 다른 일부는 그녀의 옆에 위치하고 있습니다. \n\n그녀의 뒤에는 갈색의 벽이 있으며, 그 벽 위에는 분홍색과 파란색의 점이 퍼져 있습니다. 이 점들은 그녀의 앞에 위치하고 있으며, 그녀의 얼굴과는 약간 떨어져 있습니다. \n\n이 모든 요소들은 그녀의 앞에 위치하고 있으며, 그녀는 그들 모두를 바라보고 있습니다. 그녀의 얼굴은 중립적이며, 그녀의 입술은 빨간색입니다.',
  'description': '#ELLEaskmeanything #고윤정',
  'time': '0:00:00',
  'url': 'https://youtu.be/Pb2WhqgUWl8?si=-qb_oyJeOknaa_aQ',
  'vid_title': "환혼_2_에서_이재욱과_오열한_바로_그_장면!_'희수'_vs_'낙수'_고윤정의_선택은?_고윤정의_ASK_ME_ANYTHING_____ELLE_KOREA"},
 {'_additional': {'id': 'd86fccc0-6eb3-43cd-a7c3-29104a80211b'},
  'caption': '이 이미지에는 한 여성이 주요한 역할을 하고 있습니다. 그녀는 빈 벽 앞에 서 있으며, 그녀의 옷은 검정과 흰색의 체크 패턴을 가지고 있습니다. 그녀는 목에 목걸이를 착용하고 있으며, 목걸이는 반짝이는 크리스탈로 장식되어 있습니다. 그녀의 입술은 빨간색으로 그리고, 그녀의 눈은 빛나는 눈동자를 가지고 

# DB에 존재하는 class들 확인하기

In [31]:
import weaviate
from typing import List, Dict
import json

def get_all_classes() -> List[Dict]:
    client = weaviate.Client("http://115.71.28.105:2448") 

    try:
        schema = client.schema.get()
        
        classes = schema.get('classes', [])
        
        print(f"총 {len(classes)}개의 클래스를 찾았습니다:")
        for idx, class_info in enumerate(classes, 1):
            class_name = class_info.get('class')
            description = class_info.get('description', 'No description')
            property_count = len(class_info.get('properties', []))
            
            print(f"\n{idx}. 클래스명: {class_name}")
            print(f"   설명: {description}")
            print(f"   속성 개수: {property_count}")
            
            # 속성 정보 출력 (optional)
            if property_count > 0:
                print("   속성 목록:")
                for prop in class_info.get('properties', []):
                    prop_name = prop.get('name')
                    prop_type = prop.get('dataType', [])
                    print(f"   - {prop_name}: {prop_type}")
        
        return classes

    except Exception as e:
        print(f"클래스 목록 가져오기 실패: {str(e)}")
        return []

def save_schema_to_json(classes: List[Dict], filename: str = "weaviate_schema.json"):

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(classes, f, ensure_ascii=False, indent=2)
        print(f"\n스키마 정보가 {filename}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {str(e)}")

def get_class_names() -> List[str]:
   
    client = weaviate.Client("http://115.71.28.105:2448") 
    
    try:
        schema = client.schema.get()
        class_names = [class_info['class'] for class_info in schema.get('classes', [])]
        
        print("\n존재하는 클래스 목록:")
        for idx, name in enumerate(class_names, 1):
            print(f"{idx}. {name}")
            
        return class_names
    except Exception as e:
        print(f"클래스 이름 가져오기 실패: {str(e)}")
        return []

if __name__ == "__main__":
    # print("1. 간단한 클래스 이름 목록:")
    # class_names = get_class_names()
    
    print("\n2. 상세 클래스 정보:")
    classes = get_all_classes()
    
    # # 스키마 정보 저장 (선택사항)
    # if classes:
    #     save_schema_to_json(classes)


2. 상세 클래스 정보:
총 2개의 클래스를 찾았습니다:

1. 클래스명: Captioning_retrieval
   설명: No description
   속성 개수: 4
   속성 목록:
   - time: ['text']
   - caption: ['text']
   - url: ['text']
   - vid_title: ['text']

2. 클래스명: Stt_retrieval
   설명: No description
   속성 개수: 5
   속성 목록:
   - start: ['number']
   - end: ['number']
   - text: ['text']
   - url: ['text']
   - vid_title: ['text']


/tmp/ipykernel_7442/276580988.py:6: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client("http://115.71.28.105:2448")


In [40]:
[c["class"] for c in classes]

['Captioning_retrieval', 'Stt_retrieval']

In [41]:
list(map(lambda x: x["class"], classes))

['Captioning_retrieval', 'Stt_retrieval']

In [42]:
classes

[{'class': 'Captioning_retrieval',
  'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
   'cleanupIntervalSeconds': 60,
   'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
  'multiTenancyConfig': {'autoTenantActivation': False,
   'autoTenantCreation': False,
   'enabled': False},
  'properties': [{'dataType': ['text'],
    'description': "This property was generated by Weaviate's auto-schema feature on Thu Nov 14 07:01:20 2024",
    'indexFilterable': True,
    'indexRangeFilters': False,
    'indexSearchable': True,
    'name': 'time',
    'tokenization': 'word'},
   {'dataType': ['text'],
    'description': "This property was generated by Weaviate's auto-schema feature on Thu Nov 14 07:01:20 2024",
    'indexFilterable': True,
    'indexRangeFilters': False,
    'indexSearchable': True,
    'name': 'caption',
    'tokenization': 'word'},
   {'dataType': ['text'],
    'description': "This property was generated by Weaviate's auto-schema feature on Thu Nov 14